In [1]:
import numpy as np 
import matplotlib.pyplot as plt
np.set_printoptions(precision=3, suppress=True)
np.set_printoptions(legacy='1.13')


Model Summary: A forward-looking agent manages machine maintenance over discrete states x \in \{0,1,2,3\}, choosing between skipping maintenance (d=0) or performing maintenance (d=1).
Utility includes a deterministic cost component -2x or -5, plus an i.i.d. Type I Extreme Value shock \epsilon_d, leading to logit choice probabilities.
State transitions are stochastic: skipping worsens condition with probability 0.8, while maintenance resets the machine to state 0.

In [6]:
# model fundamentals here 
beta = 0.95 

def utility(x,d):
    shock = np.random.gumbel(loc=0, scale= 1, size=1)
    if d == 1:
        return -5 + shock 
    if d == 0:
        return -2*x + shock

P = {0: np.array([
        [0.2, 0.8, 0.0, 0.0],  # from x = 0 if d = 1 (repair)
        [0.0, 0.2, 0.8, 0.0],  # from x = 1 if d = 1
        [0.0, 0.0, 0.2, 0.8],  # from x = 2 if d = 1
        [0.0, 0.0, 0.0, 1.0]   # from x = 3 if d = 1
    ]),
    1: np.array([ #surely go back to 0 
        [1.0, 0.0, 0.0, 0.0],  # from x = 0 if d = 1 (repair)
        [1.0, 0.0, 0.0, 0.0],  # from x = 1 if d = 1
        [1.0, 0.0, 0.0, 0.0],   # from x = 2 if d = 1
        [1.0, 0.0, 0.0, 0.0]   # from x = 3 if d = 1
    ])} 
#stochastic expectation of value function
def expected_value(x,d,V):
    return np.dot(P[d][x],V)  


1.0


In [3]:
# Solving: A. iteration 
V = np.zeros(4)

def iteration(V):
    last_one = V.copy()
    for i in range(4): #iterating over states
        v1 = np.exp(utility(i,1) + beta*expected_value(i,1,last_one) )
        v0 = np.exp(utility(i,0) + beta*expected_value(i,0,last_one) )
        V[i] = np.log(v1 + v0)

distance = 1
distance_record = []
first, second, third= [],[],[]

while distance > 0.001:
    first.append(V[0])
    second.append(V[1])
    third.append(V[2])
    last_one = V.copy()
    iteration(V)
    distance = sum(abs(last_one - V))
    distance_record.append(distance)

print("we iterated", len(first), "times")
print("the converged EV function is", V)


/var/folders/jb/hgpyq8d53z55039vmxnn8_4h0000gn/T/ipykernel_69343/2115496494.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  V[i] = np.log(v1 + v0)


KeyboardInterrupt: 